# [IAPR][iapr]: Lab 1 ‒  Image segmentation


**Group ID:** xx

**Author 1 (sciper):** Student Name 1 (xxxxx)  
**Author 2 (sciper):** Student Name 2 (xxxxx)   
**Author 3 (sciper):** Student Name 3 (xxxxx)   

**Release date:** 12.03.2021  
**Due date:** 26.03.2021 


## Important notes

The lab assignments are designed to teach practical implementation of the topics presented during class well as preparation for the final project, which is a practical project which ties together the topics of the course. 

As such, in the lab assignments/final project, unless otherwise specified, you may, if you choose, use external functions from image processing/ML libraries like opencv and sklearn as long as there is sufficient explanation in the lab report. For example, you do not need to implement your own edge detector, etc.

**! Before handling back the notebook !** rerun the notebook from scratch `Kernel` > `Restart & Run All`


[iapr]: https://github.com/LTS5/iapr

---
## 0. Extract relevant data
We first need to extract the `lab-01-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [1]:
import tarfile
import os
import skimage.io
import matplotlib.pyplot as plt

data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-01-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')

if not os.path.exists(tar_path):
    raise Exception('Path to is not valid {}'.format(tar_path))
    
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)

---
# FUNCTIONS

In [2]:
# Place functions that we coded below:

---
## Part 1: Brain segmentation

Your goal: compute the size of the brain (without the skull) in pixels in a 2D image of a human head taken by Magnetic Resonance Imaging (MRI) using:
* Region growing (5 pts)
* Contour detection (5 pts)
* Additional method of your choice (5 pts)

Each section should display the resulting segmenttion as well as the size in pixel of the detected region. Comment each method limitations and/or advantages.

### 1.1 Brain image visualization

In [ ]:
%matplotlib inline

# Load image
data_path = os.path.join(data_base_path, data_folder)
brain_im = skimage.io.imread(os.path.join(data_path, 'brain-slice40.tiff'))
im_h, im_w = brain_im.shape

# Display MRI image
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.imshow(brain_im, cmap='gray')
ax.set_title('MRI brain image ({} px, {} px)'.format(im_h, im_w))
ax.axis('off')
plt.show()

### 1.2 Region growing (5pts)

In [ ]:
# Add your implementation and discussion

### 1.3 Contour detection (5pts)

In [ ]:
# Add your implementation and discussion

### 1.4 Additional method (5pts)

In [ ]:
# Add your implementation and discussion

---
## Part 2: Shape/color segmentation

You will find hereafter three pictures taken under three different illuminations, containing some shapes with different colors. We ask you to create a routine to:

1. Count the number of shapes of each color (5pts).
2. Compute the total area (in pixels) of each color (5pts).

Please note that one specific challenge is to be robust to illumination changes. Therefore some kind of intensity normalization should probably be used.

### 2.1 Visualization

In [ ]:
# Load images
im_names = ['arena-shapes-01', 'arena-shapes-02', 'arena-shapes-03']
filenames = [os.path.join(data_path, name) + '.png' for name in im_names]
ic = skimage.io.imread_collection(filenames)
images = skimage.io.concatenate_images(ic)
print('Number of images: ', images.shape[0])
print('Image size: {}, {} '.format(images.shape[1], images.shape[2]))
print('Number of color channels: ', images.shape[-1])

In [ ]:
# Plot images
fig, axes = plt.subplots(1, 3, figsize=(12, 12))
for ax, im, nm in zip(axes.ravel(), images, im_names):
    ax.imshow(im)
    ax.axis('off')
    ax.set_title(nm)
plt.show()

### 2.2 Number of shapes of each color (5 pts)

In [ ]:
# Add your implementation and discussion

from scipy.signal import find_peaks
import cv2 as cv
import numpy as np

def buildMask(img, channel, limits, negative=False):
    # masks pixels with a value outside of limits
    # channel: channel to mask
    # limits[2]: intensity values of pixels. Pixels with values outside will be masked
    # negative: invert the mask
    mask = np.zeros(img[:,:,0].shape, np.uint8)
    if negative:
        mask[np.logical_and(img[:,:,2] > limits[0], img[:,:,2] < limits[1])] = 255
    else:
        mask[np.logical_or(img[:,:,2] < limits[0], img[:,:,2] > limits[1])] = 255
    return mask


def findHistRegion(hist):
    # find region in histogram where values are at least min_pixels

    # find highest peak
    peak_id = np.argmax(hist)
    peak = hist[peak_id]
    
    # find limits of peak
    low_value_found = False
    count = 0

    for i in range(peak_id, len(hist), 1):
        if hist[i] < peak*0.01:
            if count >= 10:                 # check if next 10 points are also under 1% of peak
                break
            elif low_value_found:           # increase count of low values found
                count += 1
            else:
                idx_after = i              # save idx value
                low_value_found = True      # start checking next 10 points
        elif low_value_found:               # if encounter another point above 1%, reset count
            low_value_found = False
            count = 0

    low_value_found = False
    count = 0

    for i in range(peak_id, 0, -1):
        if hist[i] < peak*0.01:
            if count >= 10:                 # check if next 10 points are also under 1% of peak
                break
            elif low_value_found:           # increase count of low values found
                count += 1
            else:
                idx_before = i              # save idx value
                low_value_found = True      # start checking next 10 points
        elif low_value_found:               # if encounter another point above 1%, reset count
            low_value_found = False
            count = 0

    return [idx_before-5, idx_after+5]      # add 5 to each side to expand peak

def findHistRegionOld(hist, min_pixels=500):
    # find region in histogram where values are at least min_pixels
    for i,n in enumerate(hist):
        if n > min_pixels:
            low_value = i
            break
    for i,n in enumerate(hist[::-1]):
        if n > min_pixels:
            high_value = len(hist)-i
            break
    return [low_value-5, high_value+5]

def findHistPeaks(hist, n_peaks, peak_distance=20):
    # find first "n_peaks" of peaks in histogram
    # peak_distance: minimum distance between two peaks
    peaks_idx,_ = find_peaks(hist.reshape(-1), distance=peak_distance)
    peaks = hist[peaks_idx].reshape(-1)
    peaks_sorted = sorted(((value, index) for index, value in zip(peaks_idx,peaks)), reverse=True)
    return [x[1] for x in peaks_sorted[:n_peaks]]         # keep only indexs of first n_peaks

def getColorRegions(color1, color2, background):
    if color1 < background[0] and color2 < background[0]:
        return [(0,(color1+color2)/2), ((color1+color2)/2+1,background[0]-1)]
    elif color1 < background[0] and color2 > background[1]:
        return [(0,background[0]-1), (background[1]+1,255)]
    elif color1 > background[1] and color2 > background[1]:
        return [(bakground[0],(color1+color2)/2), ((color1+color2)/2+1,255)]
    else:
        raise Exception('colors found not valid')

def getMasksColor(img):
    # returns the masks of two colors in the image

    # full histogram
    hist = cv.calcHist([img],[2],None,[256],[0,256])
    #plt.plot(hist)
    #plt.show

    # find limit of background intensity
    limits = findHistRegion(hist)

    # build mask to ignore background
    mask_background = buildMask(img, 2, limits)
    
    # histogram of masked image
    hist_mask = cv.calcHist([img],[2],mask_background,[256],[0,256])
    #plt.plot(hist_mask)
    #plt.show

    # get two peaks
    peak_idx = findHistPeaks(hist_mask, n_peaks = 2, peak_distance = 20)
    if peak_idx[0] > peak_idx[1]: peak_idx = peak_idx[::-1]         # sort peak_idx

    # get color regions
    color_regions = getColorRegions(peak_idx[0], peak_idx[1], limits)

    # build masks of two colors
    mask1 = buildMask(img, 2, color_regions[0], negative=True)
    mask2 = buildMask(img, 2, color_regions[1], negative=True)

    return [mask1,mask2], peak_idx


for n_image in range(3):
    # get image
    img = images[n_image].copy()

    # get the masks corresponding to each color
    masks, peak_idx = getMasksColor(img)
    mask1, mask2 = masks

    # opening and closing
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (3,3))
    mask1 = cv.morphologyEx(mask1, cv.MORPH_CLOSE, kernel)
    mask1 = cv.morphologyEx(mask1, cv.MORPH_OPEN, kernel)
    mask2 = cv.morphologyEx(mask2, cv.MORPH_CLOSE, kernel)
    mask2 = cv.morphologyEx(mask2, cv.MORPH_OPEN, kernel)

    # find contours of area big enough to avoid false contour detections
    contours1,_ = cv.findContours(mask1, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    contours_valid1 = []
    for c in contours1:
        if cv.contourArea(c) > 100:
            contours_valid1.append(c) 
    contours2,_ = cv.findContours(mask2, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
    contours_valid2 = []
    for c in contours2:
        if cv.contourArea(c) > 100:
            contours_valid2.append(c) 
    
    # compute area
    area1 = 0
    area2 = 0
    for c in contours_valid1:
        area1 += cv.contourArea(c)
    for c in contours_valid2:
        area2 += cv.contourArea(c)

    # count regoins
    print('IMAGE {}'.format(n_image))
    print('There are {} shapes of color 1 (blue intensity = {}) with an area of {} pixels'.format(len(contours_valid1), peak_idx[0], area1))
    print('There are {} shapes of color 2 (blue intensity = {}) with an area of {} pixels'.format(len(contours_valid2), peak_idx[1], area2))

    # draw contours
    mask1 = np.expand_dims(mask1, axis=2)
    mask1 = np.concatenate((mask1,mask1,mask1),axis=2)
    cv.drawContours(mask1, contours_valid1[:], -1, (0,255,0), 2)
    mask2 = np.expand_dims(mask2, axis=2)
    mask2 = np.concatenate((mask2,mask2,mask2),axis=2)
    cv.drawContours(mask2, contours_valid2[:], -1, (0,255,0), 2)

    # plot images
    fig, ax = plt.subplots(1, 3, figsize=(21, 6))
    ax[0].imshow(img, cmap='gray')
    ax[0].set_title('Original image {}'.format(n_image))
    ax[0].axis('off')
    ax[1].imshow(mask1, cmap='gray')
    ax[1].set_title('Mask of color 1')
    ax[1].axis('off')
    ax[2].imshow(mask2, cmap='gray')
    ax[2].set_title('Mask of color 2')
    ax[2].axis('off')
    plt.show()


### 2.3 Total area (in pixels) of each color (5 pts)

In [ ]:
# Add your implementation and discussion